# Llama

In [6]:
from utils.model_setup import *
from utils.dataset_setup import *
from utils.experiment_setup import *
from utils.save_results import *
from utils.state_control import *
from utils.run_experiment import *

In [7]:
from importlib import reload

In [8]:
model_identifier = 'llama'
path_to = "./data/"
api_key = 1 # With value -1 it will change every 10 iterations
execution_beggining = 6

## Setup

In [9]:
"""
Setup State
"""
use_experiment_state = True

state = create_state(-1, -1, -1, 6, 8, 0)
if use_experiment_state:
  state = retrieve_state(path_to, model_identifier)

{'Model': 2, 'Database': 2, 'Experimento': 1, 'Execucao': 6, 'MaxExecucao': 8, 'Batch': 60}


In [10]:

execute_experiment(state, model_identifier, path_to, api_key, execution_beggining)

Choosen Model: llama-3.2-90b-text-preview


 Setting model llama-3.2-90b-text-preview with API Key 1 


Dataset escolhido: aes_enem_dataset
Experimento escolhido: exp1
Create exp1
Prompt template: 
Corrija a seguinte redação, seguindo o método de avaliação do ENEM, entregue apenas a nota quantitativa entre 0 e 200 para cada competência e a nota final no seguinte formato: [x, y, z, w, t, x+y+z+w+t]. Não quero texto na resposta, apenas as notas numéricas para cada competência e o total.

Tema: <TEMA> 
Textos de suporte: <SUPORTE> 
Redação: <REDAÇÃO>
{'Model': 2, 'Database': 2, 'Experimento': 1, 'Execucao': 6, 'MaxExecucao': 8, 'Batch': 60}
{'Model': 2, 'Database': 2, 'Experimento': 1, 'Execucao': 6, 'MaxExecucao': 8, 'Batch': 60}
Saving state: {'Model': 2, 'Database': 2, 'Experimento': 1, 'Execucao': 6, 'MaxExecucao': 8, 'Batch': 60}


Execution number: 6


Folder './data/results/llama-3.2-90b-text-preview' already exists.
Folder './data/results/llama-3.2-90b-text-preview/exp1' already e

KeyError: <function run_model at 0x7fbde4f0a440>